Se tienen los Output de dos programas (Kraken y Kaiju), los cuales arrojan , c- Clasificado,  nombre del Read, y el Taxon; 

Objetivo: Hacer un scrpt que tome las salidas de varios programas  de asignacion taxonomica y los ponga en una tabla en la que : (Nombre de Read/ Taxon Asignado por Kraken/  Taxon Asignado por Kaiju) y así para lograr generalizar para varios programas.

1. Poner los datos de los outputs ordenados en la misma tabla...(Nombre/Kraken/Kaiju) 
    1. Leer un (.csv o .tsc) y convertirlo en diccionario.
    1. Hacer archivos mas pequeños de los datos
    1. Crear diccionarios para los datos
    1. Hacer un for para ordenar los datos



In [3]:
#lectura de los datos por medio de pandas, se tomaron las 10 primeras lineas de ambos outputs
import pandas as pd
#datakraken = pd.read_csv('data/EG1BMAA_kraken',sep='\t',header=None, names=["c-clasificado", "read","taxon","1","2"]) 
# pandas.leer archivo .csv(nombre del archivo, separado por tabulaciones, columnas sin nombre)
datakraken10 = pd.read_csv('data/10lineas_kraken',sep='\t',header=None, names=["c-clasificado", "read","taxon","1","2"])
#datakaiju = pd.read_csv('data/EG1BMAA_kaiju',sep='\t',header=None, names=["c-clasificado", "read","taxon"])
datakaiju10 = pd.read_csv('data/10lineas_kaiju',sep='\t',header=None, names=["c-clasificado", "read","taxon"])

In [4]:
datakraken10

,c-clasificado,read,taxon,1,2
0,C,NB501110:123:H7C3WBGX7:1:11101:9813:1065,547,151|151,A:6 2:5 1224:1 2:3 1236:10 91347:6 543:1 1236:...
1,C,NB501110:123:H7C3WBGX7:1:11101:9032:1067,329,150|150,A:6 119060:5 48736:5 329:13 0:4 329:7 0:6 1190...
2,C,NB501110:123:H7C3WBGX7:1:11101:9958:1067,69218,149|151,A:5 91347:17 69218:5 543:1 69218:21 543:4 6921...
3,C,NB501110:123:H7C3WBGX7:1:11101:22148:1069,1076549,150|150,A:6 1560356:1 91347:73 0:9 1560356:5 0:18 1560...
4,C,NB501110:123:H7C3WBGX7:1:11101:17061:1072,487316,150|151,A:6 469:57 487316:11 0:8 487316:2 0:8 469:3 0:...
5,C,NB501110:123:H7C3WBGX7:1:11101:7661:1075,69218,150|150,A:6 91347:3 547:5 91347:3 69218:14 547:1 69218...
6,C,NB501110:123:H7C3WBGX7:1:11101:3445:1076,1076549,151|151,A:6 91347:68 0:16 91347:2 0:12 91347:13 |:| 0:...
7,C,NB501110:123:H7C3WBGX7:1:11101:9350:1077,487316,151|150,A:6 0:3 469:64 487316:2 469:7 487316:22 469:13...
8,C,NB501110:123:H7C3WBGX7:1:11101:5326:1077,48736,151|151,A:6 2:111 |:| 48736:70 0:22 2:25
9,C,NB501110:123:H7C3WBGX7:1:11101:21410:1079,1560356,151|150,53335:11 91347:5 543:1 0:1 543:4 91347:3 2:6 5...


In [5]:
# como kraken10 tiene dos columnas extra que no necesitamos por el momento las eliminamos con .drop,
# asi quedan los dos outputs con las misma comunas
datakraken101 = datakraken10.drop(['c-clasificado','1', '2'], axis=1)
datakraken101

,read,taxon
0,NB501110:123:H7C3WBGX7:1:11101:9813:1065,547
1,NB501110:123:H7C3WBGX7:1:11101:9032:1067,329
2,NB501110:123:H7C3WBGX7:1:11101:9958:1067,69218
3,NB501110:123:H7C3WBGX7:1:11101:22148:1069,1076549
4,NB501110:123:H7C3WBGX7:1:11101:17061:1072,487316
5,NB501110:123:H7C3WBGX7:1:11101:7661:1075,69218
6,NB501110:123:H7C3WBGX7:1:11101:3445:1076,1076549
7,NB501110:123:H7C3WBGX7:1:11101:9350:1077,487316
8,NB501110:123:H7C3WBGX7:1:11101:5326:1077,48736
9,NB501110:123:H7C3WBGX7:1:11101:21410:1079,1560356


In [6]:
datakaiju101 = datakaiju10.drop(['c-clasificado'], axis=1)
datakaiju101

,read,taxon
0,NB501110:123:H7C3WBGX7:1:11101:22148:1069,470934
1,NB501110:123:H7C3WBGX7:1:11101:12474:1085,69218
2,NB501110:123:H7C3WBGX7:1:11101:19868:1087,53335
3,NB501110:123:H7C3WBGX7:1:11101:10733:1094,69218
4,NB501110:123:H7C3WBGX7:1:11101:20523:1097,1224136
5,NB501110:123:H7C3WBGX7:1:11101:9448:1118,470934
6,NB501110:123:H7C3WBGX7:1:11101:21936:1120,428406
7,NB501110:123:H7C3WBGX7:1:11101:25518:1127,329
8,NB501110:123:H7C3WBGX7:1:11101:22286:1152,1312959
9,NB501110:123:H7C3WBGX7:1:11101:6718:1180,53335


In [10]:
datakraken101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   read    10 non-null     object
 1   taxon   10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


In [11]:
datakaiju101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   read    10 non-null     object
 1   taxon   10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


Cree dos diccionarios, cuyas 'keys' son los reads y los 'values' son los taxones segun la clacificación.

In [53]:
dickraken = datakraken101.set_index('read').T.to_dict('list')
dickraken

{'NB501110:123:H7C3WBGX7:1:11101:9813:1065': [547],
 'NB501110:123:H7C3WBGX7:1:11101:9032:1067': [329],
 'NB501110:123:H7C3WBGX7:1:11101:9958:1067': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [1076549],
 'NB501110:123:H7C3WBGX7:1:11101:17061:1072': [487316],
 'NB501110:123:H7C3WBGX7:1:11101:7661:1075': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:3445:1076': [1076549],
 'NB501110:123:H7C3WBGX7:1:11101:9350:1077': [487316],
 'NB501110:123:H7C3WBGX7:1:11101:5326:1077': [48736],
 'NB501110:123:H7C3WBGX7:1:11101:21410:1079': [1560356]}

In [54]:
dickraken['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

[1076549]

In [55]:
dickaiju = datakaiju101.set_index('read').T.to_dict('list')
dickaiju

{'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [470934],
 'NB501110:123:H7C3WBGX7:1:11101:12474:1085': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:19868:1087': [53335],
 'NB501110:123:H7C3WBGX7:1:11101:10733:1094': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:20523:1097': [1224136],
 'NB501110:123:H7C3WBGX7:1:11101:9448:1118': [470934],
 'NB501110:123:H7C3WBGX7:1:11101:21936:1120': [428406],
 'NB501110:123:H7C3WBGX7:1:11101:25518:1127': [329],
 'NB501110:123:H7C3WBGX7:1:11101:22286:1152': [1312959],
 'NB501110:123:H7C3WBGX7:1:11101:6718:1180': [53335]}

In [56]:
dickaiju['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

[470934]

In [16]:
#merge_dicts(dicKraken, dicKaiju)

In [78]:
from itertools import chain
from collections import defaultdict
dic = defaultdict(list)
for k, v in chain(dicKraken.items(), dicKaiju.items()):
    dic[k].append(v)
dic

defaultdict(list,
            {'NB501110:123:H7C3WBGX7:1:11101:9813:1065': [[547]],
             'NB501110:123:H7C3WBGX7:1:11101:9032:1067': [[329]],
             'NB501110:123:H7C3WBGX7:1:11101:9958:1067': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [[1076549],
              [470934]],
             'NB501110:123:H7C3WBGX7:1:11101:17061:1072': [[487316]],
             'NB501110:123:H7C3WBGX7:1:11101:7661:1075': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:3445:1076': [[1076549]],
             'NB501110:123:H7C3WBGX7:1:11101:9350:1077': [[487316]],
             'NB501110:123:H7C3WBGX7:1:11101:5326:1077': [[48736]],
             'NB501110:123:H7C3WBGX7:1:11101:21410:1079': [[1560356]],
             'NB501110:123:H7C3WBGX7:1:11101:12474:1085': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:19868:1087': [[53335]],
             'NB501110:123:H7C3WBGX7:1:11101:10733:1094': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:20523:1097': [[1224

In [76]:
dic['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

[[1076549], [470934]]

In [80]:
#df = pd.DataFrame(dic,columns=['nombre_read','kraken_taxon', 'kaiju_taxon'])
#df